# Scraping Berita CNN Indonesia

## Import library yang diperlukan


Code di bawah adalah kode Python yang mengimport tiga library yaitu BeautifulSoup, requests, dan pandas. Library BeautifulSoup digunakan untuk melakukan web scraping atau mengambil data dari halaman web. Library requests digunakan untuk mengambil elemen HTML dari sebuah alamat website yang diberikan, dan ini akan menjadi input bagi Beautiful Soup untuk melakukan parsing halaman. Sedangkan library pandas digunakan untuk memanipulasi data yang telah diambil dari halaman web.

Berikut adalah penjelasan singkat mengenai tiga library tersebut:
- **BeautifulSoup**: digunakan untuk melakukan web scraping atau mengambil data dari halaman web. Library ini dapat membaca file HTML dan XML, dan mengubahnya menjadi struktur data yang mudah dibaca dan dimanipulasi.
- **requests**: digunakan untuk mengambil elemen HTML dari sebuah alamat website yang diberikan. Library ini memungkinkan kita untuk mengirimkan HTTP/1.1 requests dengan mudah.
- **pandas**: digunakan untuk memanipulasi data yang telah diambil dari halaman web. Library ini menyediakan struktur data yang fleksibel dan efisien untuk melakukan analisis data.

Kode tersebut mengimport library BeautifulSoup dengan nama alias `BeautifulSoup`, library requests tanpa alias, dan library pandas dengan nama alias `pd`. Setelah library-library tersebut diimport, kita dapat menggunakan fungsi-fungsi yang disediakan oleh masing-masing library untuk melakukan web scraping dan memanipulasi data yang telah diambil.



In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Fungsi yang digunakan untuk mengambil link navbar menggunakan function get_nav_link()

Code di bawah adalah sebuah fungsi Python yang bernama `get_nav_link()`. Fungsi ini digunakan untuk mengambil link dari menu navigasi pada halaman web CNN Indonesia.

Berikut adalah penjelasan singkat mengenai cara kerja fungsi tersebut:
- Pertama-tama, fungsi ini akan melakukan request ke halaman web CNN Indonesia menggunakan library requests.
- Setelah itu, fungsi akan memeriksa apakah respon yang diterima memiliki status code 200 atau tidak. Jika status code-nya adalah 200, maka fungsi akan melanjutkan eksekusinya. Jika tidak, maka fungsi akan mencetak pesan error.
- Selanjutnya, fungsi akan mencari elemen HTML yang memiliki tag `a` dan class `navbar__item` menggunakan library BeautifulSoup. Elemen-elemen tersebut merepresentasikan menu navigasi pada halaman web CNN Indonesia.
- Fungsi akan menyimpan link dari masing-masing menu navigasi ke dalam sebuah dictionary dengan format `{nama_menu: link_menu}`. Namun, menu dengan nama "Home" akan diabaikan.
- Setelah itu, fungsi akan melakukan update pada link dari masing-masing menu navigasi. Fungsi akan melakukan request ke halaman web yang terkait dengan link menu navigasi tersebut, dan mencari elemen HTML yang memiliki tag `a` dan atribut `dtr-act` dengan nilai "button indeks berita". Fungsi akan mengambil nilai dari atribut `href` dari elemen tersebut, dan menggunakannya sebagai link baru untuk menu navigasi tersebut.
- Setelah semua link menu navigasi telah diperbarui, fungsi akan mengembalikan dictionary yang berisi nama dan link dari masing-masing menu navigasi.




In [ ]:
def get_nav_link():
  response = requests.get('https://www.cnnindonesia.com')

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    # Get Nav
    html = response.content
    nav = BeautifulSoup(html, 'html.parser').find_all('a', class_='navbar__item')
    nav_link = {i.text.strip() : i.get('href') for i in nav if i.text.strip() not in ['Home']}

    # Update Nav Link
    for category, link in nav_link.items():
      nav_content = requests.get(link).content
      new_link = BeautifulSoup(nav_content, 'html.parser').find('a', {'dtr-act' : 'button indeks berita'}).get('href')
      nav_link[category] = new_link

    return nav_link

  else:
    print(f'Response https://www.cnnindonesia.com : {response.status_code}')

## Fungsi yang digunakan untuk mengambil data menggunakan function get_data()

Code di bawah adalah sebuah fungsi Python yang bernama `get_data(url)`. Fungsi ini digunakan untuk mengambil data dari halaman web yang diberikan melalui parameter `url`.

Berikut adalah penjelasan singkat mengenai cara kerja fungsi tersebut:
- Pertama-tama, fungsi ini akan mencetak URL yang diberikan sebagai parameter.
- Selanjutnya, fungsi akan melakukan request ke halaman web yang diberikan menggunakan library requests.
- Fungsi akan memeriksa apakah respon yang diterima memiliki status code 200 atau tidak. Jika status code-nya adalah 200, maka fungsi akan melanjutkan eksekusinya. Jika tidak, maka fungsi akan mengembalikan nilai `None`.
- Fungsi akan mencari elemen HTML yang memiliki tag `h1` dan class `mb-2 text-[28px] leading-9 text-cnn_black` menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan judul artikel pada halaman web yang diberikan.
- Fungsi juga akan mencari elemen HTML yang memiliki tag `div` dan class `text-cnn_grey text-sm mb-4` menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan tanggal publikasi artikel pada halaman web yang diberikan.
- Selanjutnya, fungsi akan mencari elemen HTML yang memiliki tag `a` dan atribut `dtr-evt` dengan nilai "breadcrumb" menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan kategori artikel pada halaman web yang diberikan.
- Fungsi akan mencari elemen HTML yang memiliki tag `div` dan class `detail-text text-cnn_black text-sm grow min-w-0` menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan isi artikel pada halaman web yang diberikan.
- Fungsi akan mengambil teks dari masing-masing elemen `p` pada elemen HTML tersebut, dan menyimpannya ke dalam sebuah string. Namun, fungsi akan mengabaikan teks yang memiliki nilai "SCROLL TO RESUME CONTENT" atau "ADVERTISEMENT".
- Setelah itu, fungsi akan menyimpan semua data yang telah diambil ke dalam sebuah list dengan format `[url, judul, tanggal, isi, kategori]`, dan mengembalikan list tersebut.



In [ ]:
def get_data(url):
  response = requests.get(url)
  if response.status_code == 200:
    data = []
    html = response.content
    content = BeautifulSoup(html, 'html.parser')
    judul = content.find('h1', class_='mb-2 text-[28px] leading-9 text-cnn_black').text.strip()
    date = content.find('div', class_='text-cnn_grey text-sm mb-4').text.strip()
    category = content.find_all('a', {'dtr-evt' : 'breadcrumb'})[1].text.strip()
    article = content.find('div', class_='detail-text text-cnn_black text-sm grow min-w-0').find_all('p')
    text = ''
    for i in article:
      if i.text.strip() not in ['SCROLL TO RESUME CONTENT', 'ADVERTISEMENT']:
        text += i.text.strip() + ' '
    data.append(url)
    data.append(judul)
    data.append(date)
    data.append(text.replace('\xa0', ''))
    data.append(category.strip())
    return data

## Fungsi yang digunakan untuk mengambil content menggunakan function get_content()

Code di atas adalah sebuah fungsi Python yang bernama `get_content(url, len_data, target)`. Fungsi ini digunakan untuk mengambil dan mengurai konten berita dari halaman web dengan URL yang diberikan. Fungsi ini akan mengambil data dari halaman web yang diberikan melalui parameter `url`, dan akan terus mengambil data dari halaman web selanjutnya hingga jumlah data yang telah diambil mencapai nilai `target`. Fungsi ini akan mengembalikan semua data yang telah diambil dalam bentuk list.

Berikut adalah penjelasan singkat mengenai cara kerja fungsi tersebut:
- Pertama-tama, fungsi ini akan membuat sebuah list kosong yang akan digunakan untuk menyimpan data yang telah diambil.
- Selanjutnya, fungsi akan melakukan request ke halaman web yang diberikan menggunakan library requests.
- Fungsi akan memeriksa apakah respon yang diterima memiliki status code 200 atau tidak. Jika status code-nya adalah 200, maka fungsi akan melanjutkan eksekusinya. Jika tidak, maka fungsi akan mencetak pesan error.
- Fungsi akan mencari elemen HTML yang memiliki tag `div` dan class `flex flex-col gap-5` menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan container yang berisi artikel-artikel pada halaman web yang diberikan.
- Fungsi akan mencari semua elemen HTML yang memiliki tag `article` dan class `flex-grow` pada container tersebut. Elemen-elemen tersebut merepresentasikan artikel-artikel pada halaman web yang diberikan.
- Fungsi akan mencari elemen HTML yang memiliki tag `a` dan atribut `dtr-act` dengan nilai "halaman selanjutnya" menggunakan library BeautifulSoup. Elemen tersebut merepresentasikan link menuju halaman web selanjutnya.
- Fungsi akan melakukan iterasi pada setiap artikel pada halaman web tersebut. Fungsi akan memeriksa apakah artikel tersebut memiliki subjudul atau tidak. Jika artikel tersebut tidak memiliki subjudul dan judulnya tidak mengandung karakter ":", maka fungsi akan mengambil link dari artikel tersebut dan memanggil fungsi `get_data()` untuk mengambil data dari halaman web tersebut. Data yang telah diambil akan disimpan ke dalam list yang telah dibuat sebelumnya.
- Fungsi akan terus melakukan iterasi pada artikel-artikel pada halaman web tersebut hingga jumlah data yang telah diambil mencapai nilai `target`. Jika jumlah data yang telah diambil sudah mencapai nilai `target`, maka fungsi akan mengembalikan semua data yang telah diambil beserta link menuju halaman web selanjutnya. Jika belum, maka fungsi akan melakukan request ke halaman web selanjutnya dan mengulangi proses yang sama.



In [ ]:
def get_content(url, len_data, target):
  data = []
  response = requests.get(url)
  if response.status_code == 200:
    html = response.content
    container = BeautifulSoup(html, 'html.parser')
    list_content = container.find('div', class_='flex flex-col gap-5').find_all('article', class_='flex-grow')
    next_button = container.find('a', {'dtr-act' : 'halaman selanjutnya'}).get('href')
    idx = 0

    while idx != len(list_content):
      subjudul = list_content[idx].find('span', class_='text-cnn_red block subjudul mb-1')
      if len(list_content[idx].find('h2').text.strip().split(':')) < 2 and subjudul == None:
        if len_data != target:
          link_content = list_content[idx].find('a').get('href')
          data_berita = get_data(link_content)
          data.append(data_berita)
          len_data += 1
          print(data_berita)
        else:
          break
          return (data, next_button)

      idx += 1

  else:
    print(f'Response {url} : {response.status_code}')
  return (data, next_button)


## Function scraping_cnn()

Fungsi `scraping_cnn(category=['Nasional'], num_data=2)` adalah sebuah fungsi Python yang digunakan untuk melakukan web scraping pada halaman web CNN Indonesia. Fungsi ini akan mengambil data dari beberapa kategori berita pada halaman web CNN Indonesia dan menyimpannya ke dalam sebuah file CSV.

Berikut adalah penjelasan singkat mengenai cara kerja fungsi tersebut:
- Pertama-tama, fungsi ini akan memanggil fungsi `get_nav_link()` untuk mengambil link dari menu navigasi pada halaman web CNN Indonesia.
- Selanjutnya, fungsi akan melakukan iterasi pada setiap kategori berita yang diberikan melalui parameter `category`. Untuk setiap kategori, fungsi akan terus mengambil data dari halaman web tersebut hingga jumlah data yang telah diambil mencapai nilai `num_data`.
- Fungsi akan memanggil fungsi `get_content()` untuk mengambil data dari halaman web yang terkait dengan kategori tersebut. Fungsi `get_content()` akan terus melakukan pengambilan data hingga jumlah data yang telah diambil mencapai nilai `num_data`.
- Setelah semua data telah diambil, fungsi akan menyimpan data tersebut ke dalam sebuah file CSV dengan nama "Data Berita CNN.csv".


In [ ]:
def scraping_cnn(category=['Nasional'], num_data=2):
  category_link = get_nav_link()
  data = []

  for i in category:
    len_data_category = 0
    while len_data_category != num_data:
      new_data, next_button = get_content(category_link[i], len_data_category, num_data)
      data += new_data
      len_data_category += len(new_data)
      category_link[i] = next_button

  print(f'Berhasil Mengambil data sebanyak {len(data)} berita dengan {len(category)} Category')

  df = pd.DataFrame(data, columns=['Url', 'Judul', 'Tanggal', 'Artikel', 'Category'])
  df.to_csv('Data Berita CNN.csv', index=False)

## Pemanggilan Fungsi scraping_cnn()

Fungsi `scraping_cnn(category=['Teknologi', 'Ekonomi', 'Olahraga'], num_data=150)` adalah sebuah fungsi Python yang digunakan untuk melakukan web scraping pada halaman web CNN Indonesia. Fungsi ini akan mengambil data dari beberapa kategori berita pada halaman web CNN Indonesia dan menyimpannya ke dalam sebuah file CSV. Fungsi ini akan mengambil 150 data untuk setiap kategori berita yang diberikan melalui parameter `category`.

Dari hasil pencarian, tidak ditemukan informasi mengenai implementasi fungsi `scraping_cnn()` yang spesifik seperti yang tertera pada pertanyaan. Namun, berdasarkan penjelasan yang telah diberikan sebelumnya, dapat disimpulkan bahwa fungsi tersebut akan melakukan web scraping pada halaman web CNN Indonesia untuk mengambil data dari beberapa kategori berita yang diberikan. Fungsi ini akan mengambil data sebanyak `num_data` untuk setiap kategori berita yang diberikan melalui parameter `category`. Setelah semua data telah diambil, fungsi akan menyimpan data tersebut ke dalam sebuah file CSV dengan nama "Data Berita CNN.csv".


In [ ]:
scraping_cnn(category=['Teknologi', 'Ekonomi', 'Olahraga'], num_data=1)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df = pd.read_csv('Data Berita CNN.csv')
df

,Url,Judul,Tanggal,Artikel,Category
0,https://www.cnnindonesia.com/teknologi/2023121...,Siapa yang Pertama Menyebar Narasi Kebencian s...,"Rabu, 13 Des 2023 17:20 WIB","Isu negatif, hoaks, hingga ujaran kebencian me...",Teknologi
1,https://www.cnnindonesia.com/teknologi/2023121...,COP28 Serukan Peralihan Penggunaan Bahan Bakar...,"Rabu, 13 Des 2023 15:51 WIB","Rancangan kesepakatan iklim PBB,COP28 pada har...",Teknologi
2,https://www.cnnindonesia.com/teknologi/2023121...,"Rekor TikTok, Pengguna Habiskan Uang Rp156 Tri...","Rabu, 13 Des 2023 14:21 WIB",Platform media sosial TikTokmemecahkan rekor s...,Teknologi
3,https://www.cnnindonesia.com/teknologi/2023121...,"Kemenangan Epic, Google Tumbang di Sidang Anti...","Rabu, 13 Des 2023 12:51 WIB","Epic Games, pembuat gim 'Fortnite' menang di p...",Teknologi
4,https://www.cnnindonesia.com/teknologi/2023121...,"Mengenal JAKI, Aplikasi yang Dipamerkan Anies ...","Rabu, 13 Des 2023 10:05 WIB",Aplikasi JAKImendapat sorotan warganet usai ca...,Teknologi
...,...,...,...,...,...
1495,https://www.cnnindonesia.com/olahraga/20231127...,Fabio Di Giannantonio Resmi Gabung VR46 di Mot...,"Senin, 27 Nov 2023 18:24 WIB",TimVR46 resmi mengumumkan perekrutan Fabio Di ...,Olahraga
1496,https://www.cnnindonesia.com/olahraga/20231127...,Bezzecchi Diklaim Lontarkan Kata-kata Kasar ke...,"Senin, 27 Nov 2023 16:44 WIB",Marc Marquezmenyebut Marco Bezzecchi melontark...,Olahraga
1497,https://www.cnnindonesia.com/olahraga/20231127...,Jadwal Prancis vs Mali di Semifinal Piala Duni...,"Senin, 27 Nov 2023 16:17 WIB",DuelPrancis U-17 vsMali akan terjadi dalam sem...,Olahraga
1498,https://www.cnnindonesia.com/olahraga/20231127...,Jadwal Argentina vs Jerman di Semifinal Piala ...,"Senin, 27 Nov 2023 15:03 WIB",Timnas Argentina U-17akan melawanJerman pada b...,Olahraga


## Cek Jumlah Data

In [ ]:
df.shape

(1500, 5)

## Cek Data Null

In [ ]:
df.isnull().sum()

Url         0
Judul       0
Tanggal     0
Artikel     0
Category    0
dtype: int64